## Загружаем данные.

In [103]:
import pandas as pd
from plotly import graph_objects as go

### https://drive.google.com/file/d/1CIoVk1p5ncHhleq3IHEeRgs9M5ANz-Iu/view?usp=sharing


In [2]:
!gdown --id 1CIoVk1p5ncHhleq3IHEeRgs9M5ANz-Iu

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1CIoVk1p5ncHhleq3IHEeRgs9M5ANz-Iu
To: /content/pfunnel_demo.csv
100% 132k/132k [00:00<00:00, 58.2MB/s]


In [3]:
df = pd.read_csv('/content/pfunnel_demo.csv')

In [4]:
%load_ext google.colab.data_table
from vega_datasets import data
df.head()

,event_name,event_time,uid
0,pageview,2019-02-27 10:16:38,99702298
1,pageview,2019-01-24 07:20:10,67118564
2,pageview,2019-05-16 06:23:10,87503951
3,pageview,2019-07-23 08:29:40,64675633
4,pageview,2019-03-11 21:47:56,73978848


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3384 entries, 0 to 3383
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   event_name  3384 non-null   object
 1   event_time  3384 non-null   object
 2   uid         3384 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 79.4+ KB


In [6]:
df['event_name'].value_counts(normalize = True) 

add_to_cart    0.358747
pageview       0.320331
checkout       0.209811
payment        0.111111
Name: event_name, dtype: float64

In [7]:
# min_check_time
users = df.pivot_table(
    index='uid', 
    columns='event_name', 
    values='event_time',
    aggfunc='min')

In [23]:
users.head(5)

event_name,add_to_cart,checkout,pageview,payment
uid,,,,
2554,2019-04-05 05:52:14,NaN,2019-04-05 05:52:32,NaN
14878,2019-05-05 16:09:51,NaN,2019-05-05 16:06:22,NaN
125581,2019-02-23 14:24:25,NaN,2019-02-23 14:25:09,NaN
129688,2019-03-10 03:51:19,NaN,2019-03-10 03:49:36,NaN
445788,2019-07-10 01:45:17,NaN,2019-07-10 01:42:28,NaN


In [31]:
step_1 = sum(~users['pageview'].isna())*1
step_2 = sum((users['pageview'].notna())  & (users['add_to_cart'] > users['pageview']))*1
step_3 = sum((users['pageview'].notna())  & (users['checkout'] > users['add_to_cart']))*1
step_4 = sum((users['pageview'].notna())  & (users['payment'] > users['checkout']))*1

In [57]:
step_4

92

Воронка с учетом времени событий.

In [43]:
fig = go.Figure(go.Funnelarea(
    text = ["Pageview","Add_to_cart", "Checkout", "Payment"],
    values = [step_1, step_2, step_3, step_4]))

fig.show()